In [3]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:85% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 1. 데이터 가져오기

In [4]:
import pandas as pd
df = pd.read_csv('C:\\AI_x\\01_download\\shareData\\부동산_250213\\전국평당분양가격(결측치제외).csv',
                encoding='cp949')
df.head()

,Unnamed: 0,지역명,연도,월,평당분양가격
0,0,서울,2013,12,18189.0
1,1,부산,2013,12,8111.0
2,2,대구,2013,12,8080.0
3,3,인천,2013,12,10204.0
4,4,광주,2013,12,6098.0


# 2. 지역명의 라벨 인코딩
- 지역명을 라벨인코딩한 지역명2(- 분석할 경우 원핫인코딩까지 할 것을 추천)

In [17]:
# 지역명 컬럼 라벨 인코딩
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['지역명2'] = encoder.fit_transform(df['지역명'])

# 결과 확인
print(df[['지역명', '지역명2']].sample(5))

     지역명  지역명2
1578  경북     3
742   충남    15
1679  전남    12
1542  전북    13
161   세종     9


# 3.  normalization 스케일 조정
- 입력변수(지역명2, 연도, 월)와 타겟변수(평당분양가격) 따로 스케일 조정(MinMaxScaler이용)
- 지역명2n, 연도n, 월n필드 추가

In [18]:
# 전체내용 :df
# 독립변수를 넘파이배열로 : df[['지역명2','연도','월']].values
# 종속변수를 넘파이배열로 : df[['평당분양가격']].values
from sklearn.preprocessing import MinMaxScaler

# 숫자형 컬럼(지역명2, 연도, 월)만 사용
X_data = df[['지역명2', '연도', '월']].values
y_data = df[['평당분양가격']].values

# 스케일러 생성
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

# 정규화 수행
X_scaled = scaler_X.fit_transform(X_data)
y_scaled = scaler_y.fit_transform(y_data)

# 결과 확인


,Unnamed: 0,지역명,연도,월,평당분양가격,지역명2,지역명2s,연도s,월s,지역명2n,연도n,월n
0,0,서울,2013,12,18189.0,8,0.000000,-1.875367,1.62196,0.5000,0.0,1.0
1,1,부산,2013,12,8111.0,7,-0.204124,-1.875367,1.62196,0.4375,0.0,1.0
2,2,대구,2013,12,8080.0,5,-0.612372,-1.875367,1.62196,0.3125,0.0,1.0
3,3,인천,2013,12,10204.0,11,0.612372,-1.875367,1.62196,0.6875,0.0,1.0
4,4,광주,2013,12,6098.0,4,-0.816497,-1.875367,1.62196,0.2500,0.0,1.0


# 4. standarization 스케일 조정
- 입력변수와 타겟변수 따로 스케일 조정(StandardScaler이용)
- 지역명2s, 연도s, 월s필드 추가

In [20]:
from sklearn.preprocessing import StandardScaler

# 입력값과 출력값 준비 (숫자형 컬럼만 사용)
X_data = df[['지역명2', '연도', '월']].values
y_data = df[['평당분양가격']].values

# 스케일러 객체 생성
scaler_Xs = StandardScaler()
scaler_ys = StandardScaler()

# 스케일 조정
X_scaled_s = scaler_Xs.fit_transform(X_data)
y_scaled_s = scaler_ys.fit_transform(y_data)

# 결과를 새로운 필드로 데이터프레임에 추가
df['지역명2s'] = X_scaled_s[:, 0]
df['연도s'] = X_scaled_s[:, 1]
df['월s'] = X_scaled_s[:, 2]

# 출력 확인
print(df[['지역명2s', '연도s', '월s']].head())
df.head()

      지역명2s       연도s       월s
0  0.000000 -1.875367  1.62196
1 -0.204124 -1.875367  1.62196
2 -0.612372 -1.875367  1.62196
3  0.612372 -1.875367  1.62196
4 -0.816497 -1.875367  1.62196


,Unnamed: 0,지역명,연도,월,평당분양가격,지역명2,지역명2s,연도s,월s,지역명2n,연도n,월n
0,0,서울,2013,12,18189.0,8,0.000000,-1.875367,1.62196,0.5000,0.0,1.0
1,1,부산,2013,12,8111.0,7,-0.204124,-1.875367,1.62196,0.4375,0.0,1.0
2,2,대구,2013,12,8080.0,5,-0.612372,-1.875367,1.62196,0.3125,0.0,1.0
3,3,인천,2013,12,10204.0,11,0.612372,-1.875367,1.62196,0.6875,0.0,1.0
4,4,광주,2013,12,6098.0,4,-0.816497,-1.875367,1.62196,0.2500,0.0,1.0


# 5. 지역명을 원핫인코딩

In [12]:
# pandas의 get_dummies를 이용한 원핫 인코딩
지역명_onehot = pd.get_dummies(df['지역명'], prefix='지역')

# 결과 확인
print(지역명_onehot.head())

   지역_강원  지역_경기  지역_경남  지역_경북  지역_광주  지역_대구  지역_대전  지역_부산  지역_서울  지역_세종  \
0      0      0      0      0      0      0      0      0      1      0   
1      0      0      0      0      0      0      0      1      0      0   
2      0      0      0      0      0      1      0      0      0      0   
3      0      0      0      0      0      0      0      0      0      0   
4      0      0      0      0      1      0      0      0      0      0   

   지역_울산  지역_인천  지역_전남  지역_전북  지역_제주  지역_충남  지역_충북  
0      0      0      0      0      0      0      0  
1      0      0      0      0      0      0      0  
2      0      0      0      0      0      0      0  
3      0      1      0      0      0      0      0  
4      0      0      0      0      0      0      0  
